In [46]:
from tensorflow.keras.datasets import mnist

In [47]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [48]:
print(x_train.shape)
print(y_train.shape)

(60000, 28, 28)
(60000,)


In [49]:
x_test.dtype

dtype('uint8')

In [ ]:
import matplotlib.pyplot as plt

: 

In [ ]:
plt.imshow(x_test[500], cmap=plt.cm.gray)

In [ ]:
import tensorflow as tf
print(tf.__version__)

#Add two layer net in Keras

In [ ]:
from tensorflow import keras

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(100, activation="sigmoid"),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model

In [ ]:
model.compile(optimizer="SGD", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
keras.utils.to_categorical(y_train, num_classes=10)

In [ ]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2])

In [ ]:
x_train.shape

In [ ]:
x_train = x_train/255.0
x_test = x_test/255.0

In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test =  keras.utils.to_categorical(y_test, num_classes=10)

In [ ]:
y_train[0]

In [ ]:
model.fit(x_train, y_train, epochs=20, batch_size=32)

In [ ]:
predictions = model.predict(x_test[0:10])

In [ ]:
import numpy as np

In [ ]:
predictions = np.argmax(predictions, axis=1)

In [ ]:
predictions

In [ ]:
labels = np.argmax(y_test[0:10], axis=1)

In [ ]:
labels

In [ ]:
predictions == labels

#LeNet

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [ ]:
class LeNet:
    def __init__(self, batch_size=32, epochs=20):
        self.batch_size = batch_size
        self.epochs = epochs
        self.model = None
        self._create_lenet()
        self._compile()
    

    def _create_lenet(self):
        self.model = Sequential([
            Conv2D(filters=6, kernel_size=(5,5), 
                   activation='sigmoid', input_shape=(28, 28, 1), 
                   padding='same'),
            AveragePooling2D(pool_size=(2, 2), strides=2),
            
            Conv2D(filters=16, kernel_size=(5,5), 
                   activation='sigmoid', 
                   padding='same'),
            AveragePooling2D(pool_size=(2, 2), strides=2),

            Flatten(),

            Dense(120, activation='sigmoid'),
            Dense(84, activation='sigmoid'),
            Dense(10, activation='softmax')
        ])

    def _compile(self):
        if self.model is None:
            print('Error: Create a model first..')
        
        self.model.compile(optimizer='Adam',
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])
        

    def _preprocess(self):
        # load mnist data
        (x_train, y_train), (x_test, y_test) = mnist.load_data()

        # normalize
        x_train = x_train/255.0
        x_test = x_test/255.0

        # add channel dim
        self.x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)  
        self.x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)  

        # one-hot encoding
        self.y_train = to_categorical(y_train, 10)
        self.y_test = to_categorical(y_test, 10)

    def train(self):
        self._preprocess()
        self.model.fit(self.x_train, self.y_train, 
                  batch_size=self.batch_size, 
                  epochs=self.epochs)

In [ ]:
lenet = LeNet(batch_size=64, epochs=10)

In [ ]:
lenet.train()

In [ ]:
predictions = np.argmax(lenet.model.predict(x_test[0:10]), axis=1)

In [ ]:
import sys
print(sys.executable)